In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Offline Hindi Voice Assistant
Optimized for Raspberry Pi 4 (4GB RAM)
CPU-only | Quantized ASR | Rule-based Intent | eSpeak TTS
"""

# ==========================================================
# IMPORT REQUIRED LIBRARIES
# ==========================================================
import os 
import sys
import time
import platform
import datetime
import subprocess
import psutil
import shutil
import warnings
import ctypes
import joblib
import onnxruntime as ort
import numpy as np
import pyaudio
import sounddevice as sd

# ==========================================================
# CONFIGURATION
# ==========================================================
INTENT_MODEL_PATH = "models/intent_model_optimized.joblib"

SAMPLE_RATE = 16000
CHANNELS = 1
CHUNK = 320  # 20 ms frames at 16kHz
VAD_MODE = 2

# ==========================================================
# REQUIRED MODELS CONFIGURATION
# ==========================================================
ASR_MODELS = {
    "coqui_stt": "models/ASR_STT/coqui_stt/coqui_stt_asr_model.tflite",
    "wav2vec2": "models/ASR_STT/IndicWav2Vec2/pytorch_model.bin",
    "vosk": "models/ASR_STT/vosk-model-small-hi-0.22/vosk-model-small-hi-0.22",
    "whisper": "models/ASR_STT/Wisper-small/model.safetensors"
}

ASR_model_select = "coqui_stt"

SYSTEM_TTS_ENGINES = {
    "espeak-ng": "espeak-ng",
    "festival": "festival"
}

TTS_model_select = "espeak-ng"

TRANSLATION_MODEL = {
    "hi-en_model": "models/Translation_model/hi-en_model"
}

# ==========================================================
# FILE SIZE UTILITY
# ==========================================================
def get_size_mb(path):
    if os.path.isfile(path):
        return round(os.path.getsize(path) / (1024**2), 2)
    elif os.path.isdir(path):
        total = 0
        for root, _, files in os.walk(path):
            for f in files:
                total += os.path.getsize(os.path.join(root, f))
        return round(total / (1024**2), 2)
    return 0

# ==========================================================
# SYSTEM CHECKS
# ==========================================================
def check_system():
    print("\n=== SYSTEM CHECK ===")
    print("Platform:", platform.platform())
    print("Python Version:", platform.python_version())
    print("CPU Cores:", psutil.cpu_count(logical=False))
    print("Logical Cores:", psutil.cpu_count())
    print("Total RAM (GB):", round(psutil.virtual_memory().total / (1024**3), 2))

    if psutil.virtual_memory().total < 3 * (1024**3):
        print("Warning: RAM may be insufficient.")

    print("System check passed.\n")

# ==========================================================
# CPU OPTIMIZATION
# ==========================================================
def optimize_cpu():
    print("Optimizing CPU usage...")

    # Limit OpenMP threads for ONNX Runtime
    os.environ["OMP_NUM_THREADS"] = "2"
    os.environ["MKL_NUM_THREADS"] = "2"

    # Pin main process to specific cores (0,1,2)
    try:
        p = psutil.Process(os.getpid())
        p.cpu_affinity([0, 1, 2])
        print("CPU affinity set.")
    except Exception as e:
        print("CPU affinity not supported:", e)

    print("CPU optimization complete.\n")

# ==========================================================
# ASR MODEL CHECK
# ==========================================================
def check_asr_models():
    print("Checking ASR models...\n")

    for name, path in ASR_MODELS.items():
        if not os.path.exists(path):
            print(f"[MISSING] {name} → {path}")
        else:
            size = get_size_mb(path)
            print(f"[OK] {name} found | Size: {size} MB")

    print("\nASR model check complete.\n")

# ==========================================================
# TTS MODEL CHECK
# ==========================================================
def check_tts_models():
    print("Checking TTS models...\n")
    for name, cmd in SYSTEM_TTS_ENGINES.items():
        if shutil.which(cmd) is None:
            print(f"[MISSING] System TTS Engine: {name}")
        else:
            print(f"[OK] {name} installed")

    print("\nTTS model check complete.\n")

# ==========================================================
# LOAD ASR MODELS
# ==========================================================
def preload_asr_model():
    print("Loading ASR model...")

    asr_model_path = ASR_MODELS.get(ASR_model_select)
    
    if ASR_model_select == "coqui_stt":
        # Load Coqui STT (TensorFlow Lite)
        try:
            import stt
            session = stt.Model(asr_model_path)
            print("Coqui STT model loaded.")
        except Exception as e:
            print(f"Error loading Coqui STT model: {e}")
            return None

    elif ASR_model_select == "wav2vec2":
        # Load Wav2Vec2 (PyTorch-based)
        try:
            model = torch.load(asr_model_path)
            print("Wav2Vec2 model loaded.")
            return model
        except Exception as e:
            print(f"Error loading Wav2Vec2 model: {e}")
            return None

    elif ASR_model_select == "vosk":
        # Load Vosk (Kaldi-based)
        try:
            model = Model(asr_model_path)
            recognizer = KaldiRecognizer(model, SAMPLE_RATE)
            print("Vosk model loaded.")
            return recognizer
        except Exception as e:
            print(f"Error loading Vosk model: {e}")
            return None

    elif ASR_model_select == "whisper":
        # Load Whisper (OpenAI)
        try:
            import whisper
            model = whisper.load_model(asr_model_path)
            print("Whisper model loaded.")
            return model
        except Exception as e:
            print(f"Error loading Whisper model: {e}")
            return None

    return None

# ==========================================================
# LOAD TTS MODELS
# ==========================================================
def preload_tts_model():
    print("Loading TTS model...")

    if TTS_model_select == "espeak-ng":
        try:
            # Check if espeak-ng is installed and available
            if shutil.which("espeak-ng") is not None:
                print("eSpeak-NG TTS model loaded.")
                return "espeak-ng"
            else:
                print("[MISSING] eSpeak-NG TTS engine not found.")
                return None
        except Exception as e:
            print(f"Error loading eSpeak-NG: {e}")
            return None

    elif TTS_model_select == "festival":
        try:
            # Check if Festival is installed and available
            if shutil.which("festival") is not None:
                print("Festival TTS model loaded.")
                return "festival"
            else:
                print("[MISSING] Festival TTS engine not found.")
                return None
        except Exception as e:
            print(f"Error loading Festival: {e}")
            return None

    return None

# ==========================================================
# MAIN INITIALIZATION
# ==========================================================
def initialize():
    warnings.filterwarnings("ignore")

    check_system()
    optimize_cpu()
    check_asr_models()
    check_tts_models()

    asr_session = preload_asr_model()
    tts_model = preload_tts_model()

    print("All model checks completed.\n")
    return asr_session, tts_model

# ==========================================================
# ENTRY POINT
# ==========================================================
if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)

    print("\n=== Starting Offline Hindi Voice Assistant ===\n")

    asr_session, tts_model = initialize()

    print("Initialization complete.")
    print("System ready.\n")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/ai4bharat/indicwav2vec-hindi.
401 Client Error. (Request ID: Root=1-6991947e-0337a2dd67b9c2c70b45854c;b0952b16-b39d-4eb2-a575-4d64bbf0a3e9)

Cannot access gated repo for url https://huggingface.co/ai4bharat/indicwav2vec-hindi/resolve/main/config.json.
Access to model ai4bharat/indicwav2vec-hindi is restricted. You must have access to it and be authenticated to access it. Please log in.

In [1]:
import os
print(os.path.exists("models/hindi_asr_int8.onnx"))


False
